# ISEL - CPS 
## Semestre 2021/22
# <br>
###  <center> Trabalho Prático 2</center>
# </br>

Trabalho realizado por:
* aluno1, numero <b> 00000 </b>
* aluno2, numero <b> 00000 </b>




A figura representa o esquema típico do processo de envio e recepção de informação usando modulações
digitais.

![fig_trab](esquemaCPS.png)

O 2º trabalho foca os blocos Codificação/descodificação do sinal e os blocos Deteção e Correção de erros. O canal é modelado pelo modelo Binary Symmetric Channel ([BSC](https://en.wikipedia.org/wiki/Binary_symmetric_channel)) 

# Codificação
Construa uma função Python que dado um Numpy array (com dimensão $N$) com valores inteiros, retorne um Numpy array (com dimensão $N.R\times 1$) com os mesmos valores convertidos para binário.
Esta função ainda deve receber como parâmetros de entrada o número de bits ($R$) a usar na conversão de cada inteiro.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

def Codific(arr_signal_quantized,R):
    """ arr_signal_quantized = [];
    arr_binary = np.zeros(len(arr_signal_quantized))
    for i in range(len(arr_signal_quantized)):
        arr_binary[i] = np.binary_repr(arr_signal_quantized[i], width=R) """

    val = "0" + str(R) + "b"
    array = list(map(lambda z: format(z,val), arr_signal_quantized))
    s = itertools.chain(*array)
    arrayint = list(map(int,s))

    arr_binary = np.array(map(int,s))
    
    return arr_binary


Construa uma função que dado um Numpy array com valores os bits, faça a conversão para inteiros (considerando R bits).

In [3]:
def Descodific(arr_binary,R):
    #resposta

    arrays = []
    x = 0
    ze = R
    for z in range(int(len(arr_binary) / R )):
        arrayconv = array[x:ze]
        arrayconv = "".join(map(str, arrayconv))
        arrays.append(arrayconv)
        x = x+R
        ze = ze + R
    arrays = list(map(lambda z: int (z,2), arrays))

    arr_signal = np.array(arrays)

    return arr_signal

Com base nas funções já implementadas, faça a codificação e descodificação PCM uniforme do sinal de áudio previamente gravado.
Verifique qual a SNR obtida e ouça o sinal descodificado. 
Use para o efeito R=3, 5 e 8.

In [4]:
#Resposta:

Faça o mesmo teste mas agora com a imagem fornecida.
comente os resultados obtidos.

In [5]:
#Resposta:

# Deteção e Correção de erros

Construa uma função que implemente o Hamming(8,4). Utilize a seguinte matriz geradora:

In [18]:
G=np.array([[1,0,0,0,1,1,0,1],[0,1,0,0,1,0,1,1],[0,0,1,0,0,1,1,1],[0,0,0,1,1,1,1,0]])
G

array([[1, 0, 0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 0]])

In [19]:
import numpy as np
import matplotlib.pyplot as plt

def Hamming_8_4(arr_bits):
    # resposta

    Ik = np.array([[1,0,1],[0,1,1],[1,1,1],[1,1,0]])
    
    
    c = np.array_split(arr_bits, 2)


    c1 = np.dot(c[0], Ik) / 2
    c2 = np.dot(c[1], Ik) / 2

    codigo = np.concatenate((c1,c2), axis=None)

    return codigo



    

Construa uma função que dado um Numpy array M bits, para cada 8 bits detecte e corriga possíveis erros, sendo o resultado 4 bits de mensagem.


In [20]:
def Detetor(arr_binary):
    #resposta

    
    
    
    return arr_bits

Faça um conjunto de testes para verificar o correto funcionamento das funções.

In [21]:
a = np.array([1,1,1,1,1,1,1,1])

Hamming_8_4(a)

array([1.5, 1.5, 1.5, 1.5, 1.5, 1.5])

Construa uma função que meça a BER entre o array de bits enviado e o array de bits recebido.

In [22]:
def BER_pratico(arr_bin_tx, arr_bin_rx):
    # resposta
    return BER_pratico

Com as duas funções realizadas e admitindo que os restantes blocos do emissor, canal e receptor possam ser simulados pela expressão $y=1*np.logical\_xor(x,np.random.binomial(1, BER_t,len(x)))$, onde $BER_t$ contém o valor pretendido para o bit error rate.

Meça o BER antes e após a correcção de erros, para diferentes valores de $BER_t$.

In [23]:
#x - array de bits a entrar no canal
#y - array de bits a sair do canal
y = 1 * np.logical_xor( x, np.random.binomial( 1, BERt, len(x) ) )

#resposta

NameError: name 'x' is not defined

Considere o sinal de áudio previamente gravado. Meça SNR na recepção, o BER antes e após a correcção de erros, para diferentes valores de  𝐵𝐸𝑅𝑡 . (considera-se que o ruído é a diferença entre o sinal à entrada do transmissor e o sinal à saída do receptor); Comente os resultados obtidos.

Considere a imagem fornecida e meça a SNR e o BER com e sem correção de erros.
Visualize a imagem no receptor e comente os resultados.